In [11]:
import cv2
import os
import shutil

In [16]:
# 1. Path to your patches and where to move “good” ones
PATCH_DIR    = "phase1"
GOOD_DIR     = "phase2"
os.makedirs(GOOD_DIR, exist_ok=True)

In [17]:
# Load Haar cascades for detecting specific features
eye_cascade   = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")

In [18]:
# Ensure cascades loaded correctly
if eye_cascade.empty() or mouth_cascade.empty():
    raise IOError("Could not load one or more cascade files.")


In [19]:
for fname in os.listdir(PATCH_DIR):
    if not fname.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    img_path = os.path.join(PATCH_DIR, fname)
    img = cv2.imread(img_path)
    if img is None:
        continue

    # Convert to grayscale and upscale slightly to detect smaller features
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (256, 256))  # Upscale to improve small feature detection

    # Detect eyes (including partial)
    eyes = eye_cascade.detectMultiScale(
        resized, scaleFactor=1.05, minNeighbors=3, minSize=(10, 10)
    )

    # Detect lips/mouth/smile/teeth
    mouth = mouth_cascade.detectMultiScale(
        resized, scaleFactor=1.1, minNeighbors=2, minSize=(10, 10)
    )

    # Delete image if any eyes or mouth features are detected
    if len(eyes) == 0 and len(mouth) == 0:
        shutil.copy(img_path, os.path.join(GOOD_DIR, fname))

print("✅ Done! Clean skin-only patches saved to:", GOOD_DIR)

✅ Done! Clean skin-only patches saved to: phase2
